In [173]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time

  # Replace with your file's ID
#list of articles
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
#list of districts and alloted funds
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
# past public data
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"


inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"
master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [174]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [175]:
def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [176]:
articles_df = read_file(article_data_id)
# Find invalid article names
def valid_articles(one):
    invalid_articles = set(one) - set(articles_df["Articles"])

    # Output result
    if invalid_articles:
        print("These articles are not in the master list:")
        print("\n".join(invalid_articles))
    else:
        print("All articles are valid.")


Download progress: 100%


## District


In [177]:
district_df = read_file(master_data_id)
district_df


Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Ariyalur,Agri Battery Sprayer,Article,7,4200.0,29400.0,176000.0,NaN,No,District
1,Ariyalur,Canon Printer 6030B (USB),Article,1,13900.0,13900.0,NaN,NaN,No,District
2,Ariyalur,Desktop Computer,Article,1,45500.0,45500.0,NaN,NaN,No,District
3,Ariyalur,Sewing Machine ORD,Article,15,5200.0,78000.0,NaN,9200.0,No,District
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520.0,160000.0,NaN,No,District
...,...,...,...,...,...,...,...,...,...,...
366,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000.0,NaN,NaN,Rameswari,District
367,Virudhunagar,Medical Aid,Aid,1,50000.0,50000.0,NaN,NaN,Karuppasamy,District
368,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000.0,NaN,0.0,Adhiparasakthi elderly aged Home,District
369,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372.0,15000.0,NaN,No,District


In [183]:
district_df.duplicated(subset=["NAME OF THE DISTRICT","REQUESTED ARTICLE","QUANTITY","COMMENTS"]).any()

np.False_

In [138]:
valid_articles(district_df["REQUESTED ARTICLE"])

All articles are valid.


In [139]:
district_df["NAME OF THE DISTRICT"].nunique()

60

In [140]:
district_df["REQUESTED ARTICLE"].nunique()

68

In [159]:
district_req = district_df.groupby("REQUESTED ARTICLE",as_index=False)["QUANTITY"].sum()

(district_req[district_req["REQUESTED ARTICLE"]=="AHUJA Radios"]["QUANTITY"].to_list()[0])

1

### Public

In [160]:
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,REQUESTED ARTICLE,Cost Per Unit,QUANTITY,Total Value,Comments,Beneficiary Type
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,15000,1,15000,Rs.15000 for children,Public
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public
...,...,...,...,...,...,...,...,...,...,...,...,...
100,P 101,M.Kumar,612081203379,No,"Ozkalur, Kancheepuram",9597058028,Business Aid,10000,1,10000,No,Public
101,P 102,M.Vinayagam,498101932330,Yes,"Ozhalur, Chenglepat",7418225869,Financial Aid,0,1,0,25% of the cost of Handicapped scooter,Public
102,P 103,D.Yuvasree,726646590112,No,"Arakonam, Ranipet",9500061246,Education Aid,100000,1,100000,No,Public
103,P 104,T.selvi,411012263584,No,"Melakalangal, Tenkasi",9751217032,Construction Aid,20000,1,20000,No,Public


In [161]:
public_df["REQUESTED ARTICLE"].nunique()

23

In [162]:
valid_articles(public_df["REQUESTED ARTICLE"])

All articles are valid.


In [163]:
public_req = public_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
public_req

,REQUESTED ARTICLE,QUANTITY
0,Bosch Electrician Kit 13 Re,1
1,Business Aid,16
2,Construction Aid,6
3,Education Aid,7
4,Electronic weighing scale,1
5,Financial Aid,5
6,Gents Cycle,4
7,Girls Cycle,4
8,Goat,2
9,Hand Sewing Machine with Motor,2


### Institution

In [164]:
inst = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})
inst_df = inst[inst["Beneficiary Type"]=="Institution"]
inst_df

Download progress: 100%


,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type
0,I 001,Cheyyur Thasildar Office,Desktop Computer,1,45500,45500,"Thasildar Office, Cheyyur, Chengalpattu",0,needs high config PC for e-seva point,Institution


In [165]:
valid_articles(inst_df["REQUESTED ARTICLE"])

All articles are valid.


In [166]:
inst_df["REQUESTED ARTICLE"].nunique()

1

In [167]:
inst_req = inst_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
inst_req

,REQUESTED ARTICLE,QUANTITY
0,Desktop Computer,1


### Others

In [168]:
oth_df = inst[inst["Beneficiary Type"]=="Others"]
oth_df

,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type


In [169]:
valid_articles(oth_df["REQUESTED ARTICLE"])

All articles are valid.


In [170]:
oth_df["REQUESTED ARTICLE"].nunique()

0

In [171]:
oth_req = oth_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
oth_req

,REQUESTED ARTICLE,QUANTITY


In [172]:
total_qty = pd.concat([district_req, public_req,inst_req,oth_req]).groupby('REQUESTED ARTICLE', as_index=False).sum()
total_qty

,REQUESTED ARTICLE,QUANTITY
0,AHUJA Radios,1
1,Agri Battery Sprayer,109
2,Agri Manual Sprayer,33
3,Agri Power Sprayer (2 STK),22
4,Agri Power Sprayer (4 STK),4
...,...,...
69,Weighing Scale+ Bicycle+Basket for Fish Vendor,1
70,Wet Grinder 2 Ltr (Hgt),8
71,Wet Grinder 3 Ltrs,2
72,Wet Grinder 5 Ltrs,4


In [122]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
final = pd.concat([district_df,public_df,inst_df]).reset_index(drop=True).groupby(["REQUESTED ARTICLE","Beneficiary Type"], as_index=False)["QUANTITY"].sum()
final

Download progress: 100%
Download progress: 100%
Download progress: 100%


,REQUESTED ARTICLE,Beneficiary Type,QUANTITY
0,AHUJA Radios,District,1
1,Agri Battery Sprayer,District,109
2,Agri Manual Sprayer,District,33
3,Agri Power Sprayer (2 STK),District,22
4,Agri Power Sprayer (4 STK),District,4
...,...,...,...
87,Wet Grinder 2 Ltr (Hgt),District,8
88,Wet Grinder 3 Ltrs,District,1
89,Wet Grinder 3 Ltrs,Public,1
90,Wet Grinder 5 Ltrs,District,4


In [124]:
district_df

,REQUESTED ARTICLE,QUANTITY,Beneficiary Type
0,Agri Battery Sprayer,7,District
1,Canon Printer 6030B (USB),1,District
2,Desktop Computer,1,District
3,Sewing Machine ORD,15,District
4,Agri Power Sprayer (2 STK),2,District
...,...,...,...
366,Ex gratia for Deceased,1,District
367,Medical Aid,1,District
368,Provision materials to Orphanages,1,District
369,Bosch Rotary Hammer GBH 220,1,District


In [127]:
print("District QTY",district_df["QUANTITY"].sum())
print("Public QTY",public_df["QUANTITY"].sum())
print("Inst&Oth QTY",inst_df["QUANTITY"].sum())
print("total qty - merge",final["QUANTITY"].sum())
print("total qty - added",district_df["QUANTITY"].sum() + public_df["QUANTITY"].sum()+inst_df["QUANTITY"].sum())

District QTY 13125
Public QTY 106
Inst&Oth QTY 1
Unique Article 13232
total qty - merge 13232
total qty - added 13232


In [128]:
# Pivot the DataFrame to create columns for each Beneficiary Type and total
pivot_df = final.pivot_table(index="REQUESTED ARTICLE",columns="Beneficiary Type",values="QUANTITY",aggfunc="sum",fill_value=0).reset_index()

# Add missing columns for Beneficiary Types and calculate the Total
pivot_df["District"] = pivot_df.get("District", 0)
pivot_df["Public"] = pivot_df.get("Public", 0)
pivot_df["Institution"] = pivot_df.get("Institution", 0)
pivot_df["Others"] = pivot_df.get("Others", 0)

pivot_df["Total"] = pivot_df[["District", "Public", "Institution", "Others"]].sum(axis=1)
pivot_df
ord_req_id = "1ou21kOkXQpL-hoaJ-11av2m7Kwk5hsif65jVOiFaU2Y"
existing_data = read_file(ord_req_id)

# Ensure required columns exist
if "Ordered Quantity" not in existing_data:
    existing_data["Ordered Quantity"] = 0
if "Remaining Quantity" not in existing_data:
    existing_data["Remaining Quantity"] = existing_data["Total"]

# Merge existing data with the new ordered quantity
updated_df = pivot_df.merge(
            existing_data[["REQUESTED ARTICLE", "Ordered Quantity", "Remaining Quantity"]],
            on="REQUESTED ARTICLE",
            how="left",
            suffixes=("", "_old")
        ).fillna({"Ordered Quantity": 0, "Remaining Quantity": lambda x: x["Total"]})

updated_df

Beneficiary Type,REQUESTED ARTICLE,District,Institution,Public,Others,Total
0,AHUJA Radios,1,0,0,0,1
1,Agri Battery Sprayer,109,0,0,0,109
2,Agri Manual Sprayer,33,0,0,0,33
3,Agri Power Sprayer (2 STK),22,0,0,0,22
4,Agri Power Sprayer (4 STK),4,0,0,0,4
...,...,...,...,...,...,...
69,Weighing Scale+ Bicycle+Basket for Fish Vendor,1,0,0,0,1
70,Wet Grinder 2 Ltr (Hgt),8,0,0,0,8
71,Wet Grinder 3 Ltrs,1,0,1,0,2
72,Wet Grinder 5 Ltrs,4,0,0,0,4


In [132]:
print("District Total",updated_df["District"].sum())
print("Public Total",updated_df["Public"].sum())
print("Institution&oth Total",updated_df["Institution"].sum())


District Total 13125


In [119]:
updated_df["Total"].sum()

np.int64(13232)